In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# Load libraries
from sklearn.dummy import DummyClassifier
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

In [2]:
# Read the CSV file using Pandas.
alldata = pd.read_csv('sberbank.csv')

# Convert the timestamp string to an integer representing the year.
def get_year(timestamp):
    return int(timestamp[:4])
alldata['year'] = alldata.timestamp.apply(get_year)

# Select the 9 input columns and the output column.
selected_columns = ['price_doc', 'year', 'full_sq', 'life_sq', 'floor', 'num_room', 'kitch_sq', 'full_all']
alldata = alldata[selected_columns]
alldata = alldata.dropna()

# Shuffle.
alldata_shuffled = alldata.sample(frac=1.0, random_state=0)

# Separate the input and output columns.
X = alldata_shuffled.drop('price_doc', axis=1)
# For the output, we'll use the log of the sales price.
Y = alldata_shuffled['price_doc'].apply(np.log)

# Split into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
m1 = DummyRegressor()
cross_validate(m1, Xtrain, Ytrain, scoring='neg_mean_squared_error')

{'fit_time': array([0.00214934, 0.00099826, 0.00144649, 0.00097919, 0.0009985 ]),
 'score_time': array([0.        , 0.        , 0.        , 0.00103354, 0.        ]),
 'test_score': array([-0.39897319, -0.37113485, -0.38083108, -0.39057156, -0.40475168])}

In [27]:
from sklearn.model_selection import GridSearchCV
gbc = GradientBoostingClassifier()

parameters = {
    "learning_rate": [0.1, 0.25, 0.5, 0.075, 0.9, 0.15, 0.2 , 0.4 , 0.6, 0.8],
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[1, 2, 3, 4, 5,6,7,8,9,10,15,20,30,40,50],
    "max_features":["log2","sqrt" ,1,2,3,4,5,6,7,8,9,11,13,15,17,19,21],
    "criterion": ["friedman_mse",  "mse"], 
    }
#passing the scoring function in the GridSearchCV
clf = GridSearchCV(gbc, parameters,  refit=False,cv=2, n_jobs=-4) # scoring=scoring,

clf.fit(Xtrain, Ytrain)

print ("Best Score: {}".format(clf.best_score_))
print ("Best params: {}".format(clf.best_params_))

Best Score: nan
Best params: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 1, 'max_features': 'log2', 'min_samples_leaf': 0.1}


In [14]:
# evaluate gradient boosting ensemble for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor
# define dataset
X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=7)
# define the model
model = GradientBoostingRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model
n_scores = cross_val_score(model, Xtrain, Ytrain, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.333 (0.010)


In [25]:
# gradient boosting ensemble for making predictions for regression
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
# define dataset
#X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=7)
# define the model
model = GradientBoostingRegressor()
# fit the model on the whole dataset
model.fit(Xtrain, Ytrain)
# make a single prediction
row = [0.20543991, -0.97049844, 1, -0.81403429, -0.60704084, -0.81403429, 0.53113006]
yhat = model.predict([row])
# summarize prediction
print('Prediction: %d' % yhat[0])

Prediction: 14


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

In [9]:
state = 12  
test_size = 0.30  
  
X_train, X_val, y_train, y_val = train_test_split(Xtrain, Ytrain,  
    test_size=test_size, random_state=state)

In [11]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

ValueError: Unknown label type: 'continuous'

In [4]:
gbc = GradientBoostingClassifier()
gbc.fit(Xtrain, Ytrain)

ValueError: Unknown label type: 'continuous'